# Aprentissage supervisé

* 1: [Loading the database](#step1)
* 2: [Calculation of all the image's signature](#step2)
* 3: [Calculation of the centers of each class(boundaries)](#step3)
* 4: [Testing the classifier](#step4)

<a id="step1"></a>
## Loading the database 

#### For the classifier: we will use 700 images for training and 210 for testing and cross-k validation after, saving each metric calculated
#### For the neural network: We will use 700 images for training, 105 for validation and 105 for testing. 